## TP-GAN (PyTorch)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.utils import save_image
from torchvision.transforms import ToTensor, Normalize, Compose
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as tt
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split
from torchvision.datasets.utils import download_url
import os
#import cv2
import math
import tarfile
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import FileLink
from IPython.display import Image

In [160]:
def lrelu(_input):
    model = nn.LeakyReLU(0.2)
    return model(_input)

def batch_norm():
    pass

def conv_2d(_input, ni, nf, ks, stride=2):
    model = nn.Conv2d(in_channels=ni, out_channels=nf, kernel_size=ks, stride=stride, padding=ks//2, bias=False)
    return model(_input)

def conv_bn(_input, ni, nf, ks, stride):
    #out1 = conv_2d(_input, ni, nf, ks, stride)
    #model2 = nn.BatchNorm2d(nf)
    #return model2(out1)
    model = nn.Sequential(nn.Conv2d(ni, nf, kernel_size=ks, stride=stride, padding=ks//2), 
                          nn.BatchNorm2d(nf)
                         )
    return model(_input)

def conv_bn_lrelu(_input, ni, nf, ks, stride):
    out1 = conv_bn(_input, ni, nf, ks, stride)
    out2 = lrelu(out1)
    return out2
    
def fc_nn(_input, input_size, output_size):
    model = nn.Sequential(nn.Flatten(), 
                          nn.Linear(input_size, output_size)
                         )
    return model(_input)

In [125]:
def res_block(_input, ni, ks=3, stride=1):
    out1 = conv_bn_lrelu(_input, ni, ni, ks, stride)
    out2 = conv_bn(out1, ni, ni, ks, stride)
    out3 = _input + out2
    out4 = lrelu(out3)
    return out4

In [169]:
def generator_global_encoder(_input):
    ni = 3
    nf = 64
    
    conv0_ = conv_2d(_input, ni, nf, ks=7, stride=1)
    conv0 = lrelu(conv0_)
    conv0r = res_block(conv0, nf, ks=7)
    
    conv1 = conv_bn_lrelu(conv0r, nf*1, nf*1, ks=5, stride=2)
    conv1r = res_block(conv1, nf*1, ks=5)
    
    conv2 = conv_bn_lrelu(conv1r, nf*1, nf*2, ks=3, stride=2)
    conv2r = res_block(conv2, nf*2, ks=3)
    
    conv3 = conv_bn_lrelu(conv2r, nf*2, nf*4, ks=3, stride=2)
    conv3r = res_block(conv3, nf*4, ks=3)
    
    conv4 = conv_bn_lrelu(conv3r, nf*4, nf*8, ks=3, stride=2)
    conv4r1 = res_block(conv4, nf*8, ks=3)
    conv4r2 = res_block(conv4r1, nf*8, ks=3)
    conv4r3 = res_block(conv4r2, nf*8, ks=3)
    conv4r4 = res_block(conv4r3, nf*8, ks=3)
    
    fc1 = fc_nn(conv4r4, 64*512, 512)
    fc2 = torch.maximum(fc1[:, 0:256], fc1[:, 256:])
    
    return fc2

In [170]:
input1 = torch.randn(48, 3, 128, 128)

In [171]:
result = generator_global_encoder(input1)
result.shape

torch.Size([48, 256])

In [168]:
result[:, 0:256].shape

torch.Size([48, 256])

In [59]:
c = res_block(input2, 16, 3)

In [60]:
c.shape

torch.Size([20, 16, 600, 50])

In [19]:
m = nn.LeakyReLU(0.1)

In [20]:
m

LeakyReLU(negative_slope=0.1)

In [25]:
input1 = torch.randn(2)

In [26]:
input1

tensor([-0.4027,  0.3916])

In [27]:
output = m(input1)

In [28]:
output

tensor([-0.0403,  0.3916])

In [46]:
m = nn.Conv2d(16, 33, 3, 2, 1)
input2 = torch.randn(20, 16, 600, 50)
output = m(input2)

In [22]:
output.shape

torch.Size([20, 33, 25, 25])

In [10]:
input2.shape

torch.Size([20, 16, 50, 100])

In [11]:
m

Conv2d(16, 33, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))

In [67]:
input2 = torch.randn(20, 16, 600, 50)
a = conv_2d(input2, 16, 33, 3)
a.shape

torch.Size([20, 33, 300, 25])

In [18]:
b = conv_bn_lrelu(input2, 16, 33, 3, 2)

In [19]:
b.shape

torch.Size([20, 33, 25, 50])